# Cleaning the dataset for Lab 6
Data source: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data


In [1]:
from ast import literal_eval
import pandas as pd

users_threshold = 10
movies_threshold = 10
filter_neutral_scores = False

In [ ]:
reviews = pd.read_csv('movies-dataset/ratings_small.csv', names=['userId', 'movieId', 'rating', 'time'], dtype={'userId': str, 'movieId': str, 'rating': int, 'time': str}, delimiter=',', engine='python', skiprows=1)
reviews.drop(['time'], axis=1, inplace=True)
if filter_neutral_scores:
    reviews = reviews[(reviews.rating >= 4.0) | (reviews.rating <= 2.0)]

reviews_no, _ = reviews.shape
users = reviews.userId.unique()
users_no = users.shape[0]
movies = reviews.movieId.unique()
movies_no = movies.shape[0]
print(f'Got {reviews_no} entries for {users_no} users and {movies_no} movies')

flag = True
while flag:
    reviews_no, _ = reviews.shape
    reviews.where(reviews.movieId.map(reviews.movieId.value_counts()).gt(movies_threshold), inplace=True)
    reviews.where(reviews.userId.map(reviews.userId.value_counts()).gt(users_threshold), inplace=True)
    reviews.dropna(axis='index', how='any', inplace=True)
    new_reviews_no, _ = reviews.shape
    users_no = reviews.userId.unique().shape[0]
    movies_no = reviews.movieId.unique().shape[0]
    if reviews_no > new_reviews_no:
        print(f'Number of entries reduced from {reviews_no} to {new_reviews_no} for {users_no} users and {movies_no} movies')
    else:
        flag = False
        print(f'Reached the final number of {reviews_no} entries for {users_no} users and {movies_no} movies')

reviews

In [ ]:
def get_key(json_str, key):
    try:
        return literal_eval(json_str).get(key)
    except:
        return None

def get_keys(json_str, key):
    try:
        return [entry.get(key) for entry in literal_eval(json_str)]
    except:
        return None


movies_metadata = pd.read_csv('movies-dataset/movies_metadata.csv')[['id', 'original_title', 'belongs_to_collection', 'genres', 'production_countries', 'release_date']]
movies_metadata.rename(columns={'id': 'movieId'}, inplace=True)
movies_metadata['collection'] = movies_metadata['belongs_to_collection'].apply(lambda x: get_key(x, 'name'))
movies_metadata.drop(['belongs_to_collection'], axis=1, inplace=True)
movies_metadata['genres'] = movies_metadata['genres'].apply(lambda x: get_keys(x, 'name'))
movies_metadata['production_countries'] = movies_metadata['production_countries'].apply(lambda x: get_keys(x, 'iso_3166_1'))
movies_metadata['release_year'] = movies_metadata['release_date'].apply(lambda x: str(x).split('-')[0] if x else None)
movies_metadata.drop(['release_date'], axis=1, inplace=True)

movies = reviews.movieId.unique()
movies_metadata = movies_metadata[movies_metadata['movieId'].isin(movies)]
movies_metadata

In [ ]:
def parse_cast(json_str):
    try:
        return [entry.get('name') for entry in literal_eval(json_str) if 'name' in entry]
    except:
        return None

def get_cast_by_jobname(json_str, jobname):
    try:
        return [entry.get('name') for entry in literal_eval(json_str) if 'name' in entry and entry['job'] == jobname]
    except:
        return None

movies_credits = pd.read_csv('movies-dataset/credits.csv', dtype={'cast': str, 'crew': str, 'id': str})
movies_credits.rename(columns={'id': 'movieId'}, inplace=True)
movies_credits = movies_credits[movies_credits['movieId'].isin(reviews.movieId.unique())]
movies_credits['actors'] = movies_credits['cast'].apply(lambda x: parse_cast(x))
movies_credits['director'] = movies_credits['crew'].apply(lambda x: get_cast_by_jobname(x, 'Director'))
movies_credits['music'] = movies_credits['crew'].apply(lambda x: get_cast_by_jobname(x, 'Original Music Composer'))
movies_credits['screenwriter'] = movies_credits['crew'].apply(lambda x: get_cast_by_jobname(x, 'Screenplay'))
movies_credits.drop(['cast', 'crew'], axis=1, inplace=True)
movies_credits

In [5]:
# save .csv files
reviews.to_csv('movies_graph/reviews.csv', index=False)
movies_metadata.to_csv('movies_graph/metadata.csv', index=False)
movies_credits.to_csv('movies_graph/credits.csv', index=False)